In [ ]:
num_output=3
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt
model = tf.keras.applications.ResNet50(weights='imagenet')
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top = False)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
preds = tf.keras.layers.Dense(num_output, activation ='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=preds)
for layer in model.layers[:175]:
    layer.trainable = False
for layer in model.layers[175:]:
    layer.trainable = True

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)


In [ ]:
train_generator = train_datagen.flow_from_directory(r"C:\Users\Pratham Nishad\Desktop\screenshot\train", 
                                                   target_size = (150, 150),
                                                   color_mode = 'rgb',
                                                   batch_size = 32,
                                                   class_mode = 'categorical',
                                                   shuffle = True)


In [ ]:
test_generator = test_datagen.flow_from_directory(r"C:\Users\Pratham Nishad\Desktop\screenshot\test", 
                                                   target_size = (150, 150),
                                                   color_mode = 'rgb',
                                                   batch_size = 5,
                                                   class_mode = 'categorical')


In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(generator = train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size, epochs = 3, validation_data=test_generator)

In [ ]:
model.save(r"C:\Users\Pratham Nishad\Desktop\models\screenshot.h5")

In [ ]:
import numpy as np
new_model = tf.keras.models.load_model(r'C:\Users\Pratham Nishad\Desktop\models\screenshot.h5')

In [ ]:
import cv2
import time
import pyautogui
run=1
# cap.set(cv2.CAP_PROP_FRAME_WIDTH,224);
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT,224);
cap= cv2.VideoCapture(0)
i=1
ss=0
face=0
back=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('C:/Users/Pratham Nishad/Desktop/image/valid/'+'face'+str(i)+'.jpg',frame)
    Sample_Image=cv2.imread('C:/Users/Pratham Nishad/Desktop/image/valid/'+'face'+str(i)+'.jpg')
    Sample_Image = tf.keras.preprocessing.image.img_to_array(Sample_Image)
    Sample_Image = np.expand_dims(Sample_Image, axis = 0)
    Sample_Image = tf.keras.applications.resnet50.preprocess_input(Sample_Image)
    predictions = new_model.predict(Sample_Image)
    if np.argmax(predictions[0])!=2:
        if np.argmax(predictions[0])==1:
            face=1
            back=0
        else:
            back=1
            face=0
        if ss!=0:
            ss=0
        if ss==5:
            ss=0
        if predictions[0][0]>predictions[0][1]:
            if run==1:
                pyautogui.keyDown('space')
                run=0
            else:
                pass
        if predictions[0][0]<predictions[0][1]:
            if run==1:
                pass
            else:
                run=1
                pyautogui.keyDown('space')
    else:
        if ss==0 and face==1:
            ss+=1
            face=0
            pyautogui.screenshot(f'C:/Users/Pratham Nishad/Desktop/screenshot_images/image{time.time()}.jpg')
            pyautogui.alert(text=f'File name is image{time.time()}.jpg', title='Done', button=["OK"])
        else:
            ss+=1
    print('Predictions:', predictions)
    i+=1
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
    try:   
        pass
#         os.remove('C:/Users/Pratham Nishad/Desktop/image/valid/'+'face'+str(i-1)+'.jpg')
    except: pass
cap.release()
cv2.destroyAllWindows()